The BSD68 dataset was adapted from K. Zhang et al (TIP, 2017) and is composed of natural
images. The noise was artificially added, allowing for quantitative comparisons with the
ground truth, one of the benchmark used in many denoising publications. Here, we check 
the performances of Noise2Void.

In [ ]:
# Imports necessary to execute the code
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tifffile
from careamics import CAREamist
from careamics.config import create_hdn_configuration
from careamics.utils.metrics import scale_invariant_psnr
from careamics_portfolio import PortfolioManager
from PIL import Image

## Import the dataset

The dataset can be directly downloaded using the `careamics-portfolio` package, which
uses `pooch` to download the data.

In [ ]:
# instantiate data portfolio manage
portfolio = PortfolioManager()

# and download the data
root_path = Path("./data")
files = portfolio.denoising.N2V_BSD68.download(root_path)

# create paths for the data
data_path = Path(root_path / "denoising-N2V_BSD68.unzip/BSD68_reproducibility_data")
train_path = data_path / "train"
val_path = data_path / "val"
test_path = data_path / "test" / "images"
gt_path = data_path / "test" / "gt"

## Visualize data

In [ ]:
# load training and validation image and show them side by side
single_train_image = tifffile.imread(next(iter(train_path.rglob("*.tiff"))))[0]
single_val_image = tifffile.imread(next(iter(val_path.rglob("*.tiff"))))[0]

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(single_train_image, cmap="gray")
ax[0].set_title("Training Image")
ax[1].imshow(single_val_image, cmap="gray")
ax[1].set_title("Validation Image")

## Train with CAREamics

The easiest way to use CAREamics is to create a configuration and a `CAREamist`. 

### Create configuration

The configuration can be built from scratch, giving the user full control over the various
parameters available in CAREamics. However, a straightforward way to create a configuration
for a particular algorithm is to use one of the convenience functions.

In [ ]:
config = create_hdn_configuration(
    experiment_name="bsd68_hdn",
    data_type="tiff",
    axes="SYX",
    z_dims=[32] * 4,
    patch_size=(128, 128),
    batch_size=64,
    num_epochs=1,
    predict_logvar="pixelwise",
    train_dataloader_params={"num_workers": 0},
    val_dataloader_params={"num_workers": 0},
    logger=None
)

print(config)

In [ ]:
config.algorithm_config.loss.reconstruction_weight = 1-0.15
config.algorithm_config.loss.kl_weight = 0.15

In [ ]:
config.algorithm_config.lr_scheduler.parameters = {"factor": 0.5, "patience": 10}
config.algorithm_config.optimizer = {"name": "Adamax", "parameters": {"lr": 0.0003}}

In [ ]:
config.algorithm_config.optimizer


In [ ]:
config.algorithm_config.lr_scheduler

### Train

A `CAREamist` can be created using a configuration alone, and then be trained by using
the data already loaded in memory.

In [ ]:
# instantiate a CAREamist
careamist = CAREamist(source=config)

# train
careamist.train(
    train_source=train_path,
    val_source=val_path,
)

## Predict with CAREamics

Prediction is done with the same `CAREamist` used for training.

In [ ]:
import tifffile

x_test = [tifffile.imread(f) for f in test_path.rglob("*.tiff")]

In [12]:
prediction = careamist.predict(
    source=test_path,
    data_type="tiff",
    axes="YX",
    tile_size=(128, 128),
    tile_overlap=(64, 64),
    mmse_count = 10,
    batch_size=1,
    dataloader_params={"num_workers": 0},
)

### Visualize the prediction

In [ ]:
plt.imshow(prediction[0].squeeze(), cmap="gray")


In [ ]:
# Show two images
noises = [tifffile.imread(f) for f in sorted(test_path.glob("*.tiff"))]
gts = [tifffile.imread(f) for f in sorted(gt_path.glob("*.tiff"))]

# images to show
images = np.random.choice(range(len(noises)), 3)

fig, ax = plt.subplots(3, 3, figsize=(15, 15))
fig.tight_layout()

for i in range(3):
    pred_image = prediction[images[i]].squeeze()
    psnr_noisy = scale_invariant_psnr(gts[images[i]], noises[images[i]])
    psnr_result = scale_invariant_psnr(gts[images[i]], pred_image)

    ax[i, 0].imshow(noises[images[i]], cmap="gray")
    ax[i, 0].title.set_text(f"Noisy\nPSNR: {psnr_noisy:.2f}")

    ax[i, 1].imshow(pred_image, cmap="gray")
    ax[i, 1].title.set_text(f"Prediction\nPSNR: {psnr_result:.2f}")

    ax[i, 2].imshow(gts[images[i]], cmap="gray")
    ax[i, 2].title.set_text("Ground-truth")

### Compute metrics

In [ ]:
psnrs = np.zeros((len(prediction), 1))

for i, (pred, gt) in enumerate(zip(prediction, gts)):
    psnrs[i] = scale_invariant_psnr(gt, pred.squeeze())

print(f"PSNR: {psnrs.mean():.2f} +/- {psnrs.std():.2f}")
print("Reported PSNR: 27.71")

## Create cover

In [ ]:
# create a cover image
im_idx = 3
cv_image_noisy = noises[im_idx]
cv_image_pred = prediction[im_idx].squeeze()

# create image
cover = np.zeros((256, 256))
(height, width) = cv_image_noisy.shape
assert height > 256
assert width > 256

# normalize train and prediction
norm_noise = (cv_image_noisy - cv_image_noisy.min()) / (
    cv_image_noisy.max() - cv_image_noisy.min()
)
norm_pred = (cv_image_pred - cv_image_pred.min()) / (
    cv_image_pred.max() - cv_image_pred.min()
)

# fill in halves
cover[:, : 256 // 2] = norm_noise[
    height // 2 - 256 // 2 : height // 2 + 256 // 2, width // 2 - 256 // 2 : width // 2
]
cover[:, 256 // 2 :] = norm_pred[
    height // 2 - 256 // 2 : height // 2 + 256 // 2, width // 2 : width // 2 + 256 // 2
]

# plot the single image
plt.imshow(cover, cmap="gray")

# save the image
im = Image.fromarray(cover * 255)
im = im.convert("L")
im.save("BSD68_Noise2Void.jpeg")

In [ ]:
# Export the model
careamist.export_to_bmz(
    path_to_archive="bsd68_n2v_model.zip",
    friendly_model_name="BSD68_N2V",
    input_array=noises[im_idx][np.newaxis, ..., :256, :256],
    authors=[{"name": "CAREamics authors", "affiliation": "Human Technopole"}],
)